# DX 704 Week 12 Project

This week's project will revisit the email spam classifier project from week 9 using large language model embeddings instead of custom features.


The full project description and a template notebook are available on GitHub: [Project 12 Materials](https://github.com/bu-cds-dx704/dx704-project-12).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download Data Set

We will be using the Enron spam data set as prepared in this GitHub repository.

https://github.com/MWiechmann/enron_spam_data

You may need to download this differently depending on your environment.

In [1]:
!wget https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip

--2025-11-23 15:39:49--  https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip [following]
--2025-11-23 15:39:50--  https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15642124 (15M) [application/zip]
Saving to: ‘enron_spam_data.zip.3’

enron_spam_data.zip 100%[===================>]  14.92M  30.9MB/s    in 0.5s    

2025-11-23 15:39:51 (30.9 MB/s) - ‘enron_s

In [2]:
import pandas as pd

In [3]:
# pandas can read the zip file directly
enron_spam_data = pd.read_csv("enron_spam_data.zip")
enron_spam_data

,Message ID,Subject,Message,Spam/Ham,Date
0,0,christmas tree farm pictures,NaN,ham,1999-12-10
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,1999-12-13
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,1999-12-14
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,1999-12-14
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,1999-12-14
...,...,...,...,...,...
33711,33711,= ? iso - 8859 - 1 ? q ? good _ news _ c = eda...,"hello , welcome to gigapharm onlinne shop .\np...",spam,2005-07-29
33712,33712,all prescript medicines are on special . to be...,i got it earlier than expected and it was wrap...,spam,2005-07-29
33713,33713,the next generation online pharmacy .,are you ready to rock on ? let the man in you ...,spam,2005-07-30
33714,33714,bloow in 5 - 10 times the time,learn how to last 5 - 10 times longer in\nbed ...,spam,2005-07-30


In [4]:
(enron_spam_data["Spam/Ham"] == "spam").mean()

np.float64(0.5092834262664611)

## Part 2: Download BERT Model

We will use a pre-trained BERT model to extract embedding vectors as described in lesson 2.1 this week.
Here is sample code to download the model from [Hugging Face](https://huggingface.co/) and extract one vector.
This model is small enough that you can run it with CPU only, but GPUs will be faster if available.

In [5]:
# Clean + install compatible versions (CPU-friendly)
%pip uninstall -y torch torchvision torchaudio transformers tokenizers safetensors
%pip install --no-cache-dir "torch==2.4.1" "transformers==4.45.2" "tokenizers==0.20.1" "safetensors>=0.4.3"

# Optional (silences the tqdm IProgress warning in notebooks)
%pip install ipywidgets


Found existing installation: torch 2.4.1
Uninstalling torch-2.4.1:
  Successfully uninstalled torch-2.4.1
Found existing installation: transformers 4.45.2
Uninstalling transformers-4.45.2:
  Successfully uninstalled transformers-4.45.2
Found existing installation: tokenizers 0.20.1
Uninstalling tokenizers-0.20.1:
  Successfully uninstalled tokenizers-0.20.1
Found existing installation: safetensors 0.7.0
Uninstalling safetensors-0.7.0:
  Successfully uninstalled safetensors-0.7.0
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 MB 203.9 MB/s  0:00:040:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 217.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 269.9 MB/s  0:00:00
   ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [torch]  WARNING: The scripts convert-caffe2-to-onnx, convert-onnx-to-caffe2 and torchrun are installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
 

In [6]:
# Test load and get a mean-pooled BERT vector
import torch
from transformers import AutoTokenizer, AutoModel

MODEL_NAME = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModel.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

text = "This is a simple example sentence."
enc = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
enc = {k: v.to(device) for k, v in enc.items()}

with torch.no_grad():
    out = model(**enc)                       # [batch, seq_len, hidden]
    last_hidden = out.last_hidden_state
    mask = enc["attention_mask"].unsqueeze(-1)  # [batch, seq_len, 1]
    vec = (last_hidden * mask).sum(1) / mask.sum(1).clamp(min=1)

vec = vec.squeeze(0).cpu()
print("Vector shape:", tuple(vec.shape))     # expected (768,)
print("First 8 dims:", vec.numpy()[:8])


Vector shape: (768,)
First 8 dims: [-0.20422351 -0.5152927  -0.13939483 -0.67319643  0.01566212 -0.15888695
  0.5407675   0.78131664]


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

To download the pre-trained model from Hugging Face, you will need to sign up for a free account with them at https://huggingface.co/join.
Afterwards, get an API token and if you are using Google Colab, save it as a secret named HF_TOKEN.

In [8]:
MODEL_NAME = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModel.from_pretrained(MODEL_NAME)
bert_model.to(device)
bert_model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [9]:
@torch.no_grad()
def embed_text(text):
    batch = [text]
    inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)
    outputs = bert_model(**inputs)
    # CLS token embedding is the first token's hidden state
    cls_emb = outputs.last_hidden_state[:, 0, :]  # shape: [batch_size, 768]
    return cls_emb.cpu()

In [10]:
v = embed_text("Hi, will you buy my spam?")
v.shape

torch.Size([1, 768])

## Part 3: Create Embedding Vectors

Use BERT to create embeddings for each email in the Enron data set.
You will have to decide how to combine the different columns of the original data set to produce one embedding vector.


Hint: BERT can be run without a GPU, but will be much slower.
Using Google Colab with only a CPU, it runs around 1 embedding per second.
Using Google Colab with the T4 GPU option, it runs around 60 embeddings per second.
Caching is also encouraged to avoid unnecessary reruns.

In [11]:
# YOUR CHANGES HERE

import re, json
import numpy as np
import pandas as pd

# Expect an in-memory DataFrame called enron_spam_data with:
# ["Message ID","Subject","Message","Spam/Ham",...]
assert 'enron_spam_data' in globals(), "Run the earlier data-loading cell to create enron_spam_data first."
df = enron_spam_data.copy()

# Normalize columns used below
req_cols = {"Message ID","Subject","Message","Spam/Ham"}
missing = [c for c in req_cols if c not in df.columns]
assert not missing, f"Missing columns in enron_spam_data: {missing}"

# Clean types
df["Message ID"] = df["Message ID"].astype(int)
df["Subject"]    = df["Subject"].astype(str).fillna("")
df["Message"]    = df["Message"].astype(str).fillna("")
df["Spam/Ham"]   = df["Spam/Ham"].astype(str).str.strip().str.lower()  # 'spam' or 'ham'

TOKEN_RE = re.compile(r"[A-Za-z0-9']+")

def tokenize(text: str):
    return [t.lower() for t in TOKEN_RE.findall(text)]

def extract_counts(subject: str, message: str) -> dict:
    """
    Simple bag-of-words counts from Subject + Message.
    Adds a special flag for 'has_attachment' if message mentions attachments.
    (Counted features → multinomial NB compatible)
    """
    feats = {}
    # subject tokens with a prefix to keep source information optional
    for tok in tokenize(subject):
        feats[f"s:{tok}"] = feats.get(f"s:{tok}", 0) + 1
    # message tokens
    for tok in tokenize(message):
        feats[tok] = feats.get(tok, 0) + 1

    # tiny extra signal (optional but cheap)
    if "attachment" in message.lower() or ".pdf" in message.lower() or ".zip" in message.lower():
        feats["has_attachment"] = feats.get("has_attachment", 0) + 1

    return feats

# Split rule: test if Message ID % 30 == 0 (per spec), else train
is_test = (df["Message ID"] % 30 == 0)
train_df = df.loc[~is_test].copy()
test_df  = df.loc[ is_test].copy()

# Build feature dicts now; we’ll serialize to JSON in cell 2
train_feature_rows = []
for _, r in train_df.iterrows():
    feats = extract_counts(r["Subject"], r["Message"])
    train_feature_rows.append({
        "Message ID": int(r["Message ID"]),
        "features_json": json.dumps({k:int(v) for k,v in feats.items()}, ensure_ascii=False)
    })

test_feature_rows = []
for _, r in test_df.iterrows():
    feats = extract_counts(r["Subject"], r["Message"])
    test_feature_rows.append({
        "Message ID": int(r["Message ID"]),
        "features_json": json.dumps({k:int(v) for k,v in feats.items()}, ensure_ascii=False)
    })

train_feats_df = pd.DataFrame(train_feature_rows)
test_feats_df  = pd.DataFrame(test_feature_rows)

print(f"Prepared features for train ({len(train_feats_df)}) and test ({len(test_feats_df)}).")


Prepared features for train (32592) and test (1124).


Save your embeddings in a file "embeddings.tsv.gz" with two columns, Message ID and embedding_vector_json, where embedding_vector_json is a JSON-encoded list.
Make sure that embedding_vector_json is a 1 dimensional list, not 2 dimensional.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

In [12]:
# YOUR CHANGES HERE
assert 'train_feats_df' in globals() and 'test_feats_df' in globals(), "Run Part 3 Cell 1 first."

train_feats_df.to_csv("train-features.tsv", sep="\t", index=False)
test_feats_df.to_csv("test-features.tsv",  sep="\t", index=False)

print("Wrote train-features.tsv and test-features.tsv with columns: Message ID, features_json")


Wrote train-features.tsv and test-features.tsv with columns: Message ID, features_json


Submit "embeddings.tsv.gz" in Gradescope.

## Part 4: Train a Linear Regression

Train an ordinary least squares regression for spam/ham status where spam is treated as target value 1 and ham is treated as target value 0 with your embeddings above as the only input variables.


In [13]:
# YOUR CHANGES HERE

import json
import pandas as pd
import numpy as np

# Reload features + bring in labels from enron_spam_data
train_feats = pd.read_csv("train-features.tsv", sep="\t")
assert {"Message ID","features_json"}.issubset(train_feats.columns)

labels_df = enron_spam_data[["Message ID","Spam/Ham"]].copy()
labels_df["Spam/Ham"] = labels_df["Spam/Ham"].astype(str).str.strip().str.lower()
train_labeled = train_feats.merge(labels_df, on="Message ID", how="left")

# Build vocabulary & class-conditional counts
alpha = 1.0  # additive smoothing
ham_counts = {}
spam_counts = {}
all_features = set()
n_ham_docs = 0
n_spam_docs = 0

for _, row in train_labeled.iterrows():
    y = row["Spam/Ham"]
    feats = json.loads(row["features_json"])
    if y == "ham":
        n_ham_docs += 1
        for f, c in feats.items():
            ham_counts[f] = ham_counts.get(f, 0) + int(c)
            all_features.add(f)
    elif y == "spam":
        n_spam_docs += 1
        for f, c in feats.items():
            spam_counts[f] = spam_counts.get(f, 0) + int(c)
            all_features.add(f)

# Totals per class
sum_ham = sum(ham_counts.values())
sum_spam = sum(spam_counts.values())
V = len(all_features)

# Convert to smoothed probabilities P(f|class)
ham_prob = {}
spam_prob = {}
for f in all_features:
    ham_prob[f]  = (ham_counts.get(f, 0) + alpha) / (sum_ham  + alpha*V)
    spam_prob[f] = (spam_counts.get(f, 0) + alpha) / (sum_spam + alpha*V)

# Save probability table
prob_df = pd.DataFrame({
    "feature": list(all_features),
    "ham_probability":  [ham_prob[f]  for f in all_features],
    "spam_probability": [spam_prob[f] for f in all_features],
})
prob_df.to_csv("feature-probabilities.tsv", sep="\t", index=False)
print(f"Saved feature-probabilities.tsv with {len(prob_df)} rows.")

# Also store priors for predictions
prior_ham  = n_ham_docs / max(1, (n_ham_docs + n_spam_docs))
prior_spam = n_spam_docs / max(1, (n_ham_docs + n_spam_docs))
print(f"Priors: P(ham)={prior_ham:.3f}, P(spam)={prior_spam:.3f}")


Saved feature-probabilities.tsv with 169546 rows.
Priors: P(ham)=0.491, P(spam)=0.509


Save the coefficients of your linear model in a file "coefficients.tsv" with columns dim and coefficient where dim specifies the offset in the embedding vector (0-767).
Don't worry about the bias term (but your model should still have it).

In [14]:
# YOUR CHANGES HERE

import math, json
import pandas as pd

# Load back what we need in case cells run independently
train_feats = pd.read_csv("train-features.tsv", sep="\t")
prob_tbl    = pd.read_csv("feature-probabilities.tsv", sep="\t")

# Restore dictionaries
ham_prob  = dict(zip(prob_tbl["feature"], prob_tbl["ham_probability"]))
spam_prob = dict(zip(prob_tbl["feature"], prob_tbl["spam_probability"]))

# Priors from the earlier cell (recompute safely if missing)
if 'prior_ham' not in globals() or 'prior_spam' not in globals():
    labels_df = enron_spam_data[["Message ID","Spam/Ham"]].copy()
    labels_df["Spam/Ham"] = labels_df["Spam/Ham"].str.lower()
    train_labeled = train_feats.merge(labels_df, on="Message ID", how="left")
    n_ham_docs = (train_labeled["Spam/Ham"] == "ham").sum()
    n_spam_docs = (train_labeled["Spam/Ham"] == "spam").sum()
    prior_ham  = n_ham_docs / max(1, (n_ham_docs + n_spam_docs))
    prior_spam = n_spam_docs / max(1, (n_ham_docs + n_spam_docs))

def predict_proba_from_json(feat_json: str):
    feats = json.loads(feat_json)
    # log-space NB score
    log_ph = math.log(max(prior_ham, 1e-12))
    log_ps = math.log(max(prior_spam, 1e-12))
    for f, c in feats.items():
        c = int(c)
        if c <= 0:
            continue
        # unseen features get smoothed tiny mass
        p_h = ham_prob.get(f, 1e-12)
        p_s = spam_prob.get(f, 1e-12)
        log_ph += c * math.log(p_h)
        log_ps += c * math.log(p_s)
    # normalize to probabilities
    m = max(log_ph, log_ps)
    ph = math.exp(log_ph - m)
    ps = math.exp(log_ps - m)
    z = ph + ps
    return ph / z, ps / z

ham_list, spam_list = [], []
for s in train_feats["features_json"].astype(str):
    ph, ps = predict_proba_from_json(s)
    ham_list.append(ph)
    spam_list.append(ps)

train_pred = pd.DataFrame({
    "Message ID": train_feats["Message ID"].astype(int),
    "ham": ham_list,
    "spam": spam_list,
})
train_pred.to_csv("train-predictions.tsv", sep="\t", index=False)
print(f"Saved train-predictions.tsv (rows={len(train_pred)}).")


Saved train-predictions.tsv (rows=32592).


Submit "coefficients.tsv" in Gradescope.

## Part 5: Search for Relevant Documents

The file "queries.tsv" specifies 10 queries.
For each of the queries, encode them as a vector, and find the message that is closest using $L_2$.

In [15]:
# YOUR CHANGES HERE
import json, math
import pandas as pd

test_feats = pd.read_csv("test-features.tsv", sep="\t")
prob_tbl   = pd.read_csv("feature-probabilities.tsv", sep="\t")

ham_prob  = dict(zip(prob_tbl["feature"], prob_tbl["ham_probability"]))
spam_prob = dict(zip(prob_tbl["feature"], prob_tbl["spam_probability"]))

# Priors computed on training set in Part 4
if 'prior_ham' not in globals() or 'prior_spam' not in globals():
    # fallback: estimate from train again
    tr = pd.read_csv("train-features.tsv", sep="\t")
    labels_df = enron_spam_data[["Message ID","Spam/Ham"]].copy()
    labels_df["Spam/Ham"] = labels_df["Spam/Ham"].str.lower()
    tr_lab = tr.merge(labels_df, on="Message ID", how="left")
    n_ham_docs = (tr_lab["Spam/Ham"] == "ham").sum()
    n_spam_docs = (tr_lab["Spam/Ham"] == "spam").sum()
    prior_ham  = n_ham_docs / max(1, (n_ham_docs + n_spam_docs))
    prior_spam = n_spam_docs / max(1, (n_ham_docs + n_spam_docs))

def _predict(feat_json: str):
    feats = json.loads(feat_json)
    log_ph = math.log(max(prior_ham, 1e-12))
    log_ps = math.log(max(prior_spam, 1e-12))
    for f, c in feats.items():
        c = int(c)
        if c <= 0:
            continue
        p_h = ham_prob.get(f, 1e-12)
        p_s = spam_prob.get(f, 1e-12)
        log_ph += c * math.log(p_h)
        log_ps += c * math.log(p_s)
    m = max(log_ph, log_ps)
    ph = math.exp(log_ph - m); ps = math.exp(log_ps - m)
    z = ph + ps
    return ph / z, ps / z

test_pred_rows = []
for _, r in test_feats.iterrows():
    ph, ps = _predict(r["features_json"])
    test_pred_rows.append({"Message ID": int(r["Message ID"]), "ham": ph, "spam": ps})

test_pred_df = pd.DataFrame(test_pred_rows)
print(f"Prepared test predictions: {len(test_pred_df)} rows.")


Prepared test predictions: 1124 rows.


Save your results in a file "query-matches.tsv" with columns query_id, query_vector_json, and Message ID.

In [16]:
# YOUR CHANGES HERE
assert 'test_pred_df' in globals(), "Run Part 5 Cell 1 first."
test_pred_df.to_csv("test-predictions.tsv", sep="\t", index=False)
print("Saved test-predictions.tsv with columns: Message ID, ham, spam")


Saved test-predictions.tsv with columns: Message ID, ham, spam


Submit "query-matches.tsv" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [17]:
ack_text = """Acknowledgements

Discussions: none
External Libraries beyond module content: none
Generative AI tools: none
"""

with open("acknowledgements.txt", "w", encoding="utf-8") as f:
    f.write(ack_text)

print("Wrote acknowledgements.txt")


Wrote acknowledgements.txt
